In [23]:
import crud_operations as co
import sqlparse
from itertools import islice
import re
import pandas as pd

In [24]:
def processQueries():
    db = co.MySQLDB('pl_queries')
    
    queries = {}
    res = db.readAllQueries()
    for q in res:
        queries[(q[0], q[1], q[2])] = q[3]
    return queries

In [25]:
queries = processQueries()

In [26]:
len(queries)

1418

In [27]:
# Try with one query
curr = queries[(1,313665,4388082)]
parsed = sqlparse.parse(curr)
print(parsed[0])
print(parsed[0].tokens)

for tok in parsed[0].tokens:
    # print(tok.ttype, type(tok))
    if (type(tok) == sqlparse.sql.Identifier):
        print(tok.value)

select e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle
from Employee e natural join Department d
where (d.deptLocation = 'Dubai' or d.deptName = 'Sales') and
        (e.annualSalary >= 7000 and e.annualSalary <= 100000) and
        (e.empName like '_a%' or e.empName like '_i%')
order by e.empName asc, d.deptName desc
[<DML 'select' at 0x112C1CD00>, <Whitespace ' ' at 0x112C1CE50>, <IdentifierList 'e.empN...' at 0x10A330AD0>, <Newline ' ' at 0x112C1CDE0>, <Keyword 'from' at 0x10A3B0830>, <Whitespace ' ' at 0x10A3B0910>, <Identifier 'Employ...' at 0x10A3304D0>, <Whitespace ' ' at 0x10A3B0AD0>, <Keyword 'natura...' at 0x10A3B08A0>, <Whitespace ' ' at 0x10A3B0BB0>, <Identifier 'Depart...' at 0x10A3305D0>, <Newline ' ' at 0x10A3B0B40>, <Where 'where ...' at 0x112C17D50>, <Keyword 'order ...' at 0x10A336280>, <Whitespace ' ' at 0x10A3362F0>, <IdentifierList 'e.empN...' at 0x10A330050>]
Employee e
Department d


In [28]:
# For first ten queries, let's substitute:
def parse(queries):
    aliases = ['a', 'b', 'c', 'd', 'e', 'f']
    parsed_queries = []
    f = open("parsed_queries.csv", "w")
    df1 = pd.DataFrame(columns=['queryNum','queryID','variantID','queryText'])
    for query in queries:
        #print("New Query\n")
        content = query[1]
        #print(content)
        curr_alias = 0
        
        parsed = sqlparse.parse(query[1])
        
        #if len(parsed) == 1:
        prev_keyword = ""
        for tok in parsed[0].tokens:
            # print(type(tok), tok.ttype, tok.value)
            if(str(tok.ttype) == "Token.Keyword"):
                prev_keyword = tok.value.lower()
            if(type(tok) == sqlparse.sql.Identifier) and (prev_keyword == "from" or "join" in prev_keyword):
                # Process tok.value
                # The order of Department vs. Employee shouldn't matter
                alias = tok.value.split(" ")
                #print(alias)
                if(len(alias) > 1 and (alias[-1].lower != "asc" or alias[-1].lower != "desc")):
                    content = content.replace(" " + alias[-1] + " ", " " + aliases[curr_alias] + " ")
                    content = content.replace(alias[-1] + ".", aliases[curr_alias] + ".")
                    curr_alias += 1

            if(type(tok) == sqlparse.sql.IdentifierList):
                # use regex to make sure is alias one
                if re.match(r'[a-zA-Z ]+ [a-zA-Z ]+,*', tok.value):
                    sets = tok.value.split(",")
                    for s in sets:
                        alias = s.strip().split(" ")
                        #print(alias)
                        if(len(alias) > 1 and (alias[-1].lower() != "asc" and alias[-1].lower() != "desc" and "." not in alias[-1])):
                            if len(alias) == 2:
                                content = content.replace(" " + alias[-1] + ",", " " + aliases[curr_alias] + ",")
                                content = content.replace(" " + alias[-1] + "\n", " " + aliases[curr_alias] + "\n")
                                content = content.replace(alias[-1] + ".", aliases[curr_alias] + ".")
                            else:
                                as_flag = False
                                for i in alias:
                                    if as_flag:
                                        # this is an alias
                                        content = content.replace(" " + i + ",", " " + aliases[curr_alias] + ",")
                                        content = content.replace(" " + i + "\n", " " + aliases[curr_alias] + "\n")
                                        content = content.replace(i + ".", aliases[curr_alias] + ".")
                                        as_flag = False
                                    if i == "as":
                                        as_flag = True
                            curr_alias += 1
        f.write(str(query[0][0]) + ", " + str(query[0][1]) + "," + str(query[0][2]) + "," + str(content))
        new_row = {'queryNum':str(query[0][0]),'queryID': str(query[0][1]), 'variantID':str(query[0][2]), 'queryText':str(content)}
                #append row to the dataframe
        df1 = df1.append(new_row, ignore_index=True)
        f.write("\n")
        
        parsed_queries.append([query[0][0], query[0][1], query[0][2], content])
                    
        #else:
        #    print("more than one")
        #    for p in parsed:
        #        print(p, "boop")
        print("=============")
    df1.to_csv('parsed_queries_from_df.csv',index=False)
    return parsed_queries

In [29]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [30]:
n = 10
subset = take(n, queries.items())
print(subset)

[((1, 313665, 4388082), "select e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle\nfrom Employee e natural join Department d\nwhere (d.deptLocation = 'Dubai' or d.deptName = 'Sales') and\n        (e.annualSalary >= 7000 and e.annualSalary <= 100000) and\n        (e.empName like '_a%' or e.empName like '_i%')\norder by e.empName asc, d.deptName desc"), ((2, 34820, 4388082), "SELECT e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle\nFROM Employee e join Department d on e.deptID = d.deptID \nWHERE ((d.deptLocation = 'Dubai') OR (d.deptName = 'Sales')) AND (e.annualSalary >= 84000 AND e.annualSalary <= 120000) AND ((e.empName LIKE '_A%' )OR (e.empName LIKE '_I%'))\nORDER BY e.empName,  d.deptName DESC\n\n"), ((3, 325288, 4388082), "SELECT e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle\nFROM Employee e NATURAL JOIN Department d\nWHERE (d.deptLocation = 'Dubai' OR d.deptName = 'Sales') AND e.annualSalary >= 84000 AND e.annualSalary <= 120000

In [31]:
parse(subset)

[[1,
  313665,
  4388082,
  "select a.empName, a.annualSalary, b.deptName, b.deptLocation, a.jobTitle\nfrom Employee a natural join Department d\nwhere (b.deptLocation = 'Dubai' or b.deptName = 'Sales') and\n        (a.annualSalary >= 7000 and a.annualSalary <= 100000) and\n        (a.empName like '_a%' or a.empName like '_i%')\norder by a.empName asc, b.deptName desc"],
 [2,
  34820,
  4388082,
  "SELECT a.empName, a.annualSalary, b.deptName, b.deptLocation, a.jobTitle\nFROM Employee a join Department b on a.deptID = b.deptID \nWHERE ((b.deptLocation = 'Dubai') OR (b.deptName = 'Sales')) AND (a.annualSalary >= 84000 AND a.annualSalary <= 120000) AND ((a.empName LIKE '_A%' )OR (a.empName LIKE '_I%'))\nORDER BY a.empName,  b.deptName DESC\n\n"],
 [3,
  325288,
  4388082,
  "SELECT a.empName, a.annualSalary, b.deptName, b.deptLocation, a.jobTitle\nFROM Employee a NATURAL JOIN Department d\nWHERE (b.deptLocation = 'Dubai' OR b.deptName = 'Sales') AND a.annualSalary >= 84000 AND a.annualSa

In [12]:
queries

{(1,
  313665,
  4388082): "select e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle\nfrom Employee e natural join Department d\nwhere (d.deptLocation = 'Dubai' or d.deptName = 'Sales') and\n        (e.annualSalary >= 7000 and e.annualSalary <= 100000) and\n        (e.empName like '_a%' or e.empName like '_i%')\norder by e.empName asc, d.deptName desc",
 (2,
  34820,
  4388082): "SELECT e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle\nFROM Employee e join Department d on e.deptID = d.deptID \nWHERE ((d.deptLocation = 'Dubai') OR (d.deptName = 'Sales')) AND (e.annualSalary >= 84000 AND e.annualSalary <= 120000) AND ((e.empName LIKE '_A%' )OR (e.empName LIKE '_I%'))\nORDER BY e.empName,  d.deptName DESC\n\n",
 (3,
  325288,
  4388082): "SELECT e.empName, e.annualSalary, d.deptName, d.deptLocation, e.jobTitle\nFROM Employee e NATURAL JOIN Department d\nWHERE (d.deptLocation = 'Dubai' OR d.deptName = 'Sales') AND e.annualSalary >= 84000 AND e.annualSalary 

In [22]:
par_queries

[[1,
  313665,
  4388082,
  "select a.empName, a.annualSalary, b.deptName, b.deptLocation, a.jobTitle\nfrom Employee a natural join Department d\nwhere (b.deptLocation = 'Dubai' or b.deptName = 'Sales') and\n        (a.annualSalary >= 7000 and a.annualSalary <= 100000) and\n        (a.empName like '_a%' or a.empName like '_i%')\norder by a.empName asc, b.deptName desc"],
 [2,
  34820,
  4388082,
  "SELECT a.empName, a.annualSalary, b.deptName, b.deptLocation, a.jobTitle\nFROM Employee a join Department b on a.deptID = b.deptID \nWHERE ((b.deptLocation = 'Dubai') OR (b.deptName = 'Sales')) AND (a.annualSalary >= 84000 AND a.annualSalary <= 120000) AND ((a.empName LIKE '_A%' )OR (a.empName LIKE '_I%'))\nORDER BY a.empName,  b.deptName DESC\n\n"],
 [3,
  325288,
  4388082,
  "SELECT a.empName, a.annualSalary, b.deptName, b.deptLocation, a.jobTitle\nFROM Employee a NATURAL JOIN Department d\nWHERE (b.deptLocation = 'Dubai' OR b.deptName = 'Sales') AND a.annualSalary >= 84000 AND a.annualSa

In [32]:
par_queries = parse(queries.items())

In [192]:
parse([list(queries.items())[-2]])

more than one
SELECT p1.ProductName, COUNT(p2.YearReleased) - 1
FROM Products as p1, Products as p2
WHERE (p1.BrandName = p2.BrandName AND p2.YearReleased < p1.YearReleased) OR
      (p1.ProductName = p2.ProductName)
GROUP BY p1.ProductName; boop

 boop


[]

In [179]:
par_queries[0]

(1,
 "select a.empName, a.annualSalary, b.deptName, b.deptLocation, a.jobTitle\nfrom Employee a natural join Department d\nwhere (b.deptLocation = 'Dubai' or b.deptName = 'Sales') and\n        (a.annualSalary >= 7000 and a.annualSalary <= 100000) and\n        (a.empName like '_a%' or a.empName like '_i%')\norder by a.empName asc, b.deptName desc")

In [198]:
par_queries[-2]

(1417,
 'SELECT a.ProductName, COUNT(b.YearReleased) - 1\nFROM Products as a, Products as b\nWHERE (a.BrandName = b.BrandName AND b.YearReleased < a.YearReleased) OR\n      (a.ProductName = b.ProductName)\nGROUP BY a.ProductName;\n')

In [196]:
len(par_queries)

1418